<a href="https://colab.research.google.com/github/RajKumarPandey/qml_notebooks/blob/main/tfq_pki_qml_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.15.0 tensorflow-quantum==0.7.3 cirq==1.3.0

import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("https://raw.githubusercontent.com/RajKumarPandey/qml_notebooks/main/synthetic_pki_dataset.csv")
X = df.drop("label", axis=1).values
y = df["label"].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled[:, :1]
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

qubit = cirq.GridQubit(0, 0)
theta = sympy.Symbol("theta")

def create_quantum_circuit(x):
    circuit = cirq.Circuit()
    circuit.append(cirq.rx(float(x))(qubit))
    circuit.append(cirq.rx(theta)(qubit))
    return circuit

train_circuits = [create_quantum_circuit(x[0]) for x in X_train]
test_circuits = [create_quantum_circuit(x[0]) for x in X_test]

train_x = tfq.convert_to_tensor(train_circuits)
test_x = tfq.convert_to_tensor(test_circuits)

model = tf.keras.Sequential([
    tfq.layers.PQC(cirq.Circuit(cirq.rx(theta)(qubit)), cirq.Z(qubit))
])
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='mse', metrics=['accuracy'])
model.fit(train_x, y_train, batch_size=4, epochs=20)
loss, acc = model.evaluate(test_x, y_test)
print("Test accuracy:", acc)